# Run Model Bias Analysis with SageMaker Clarify (Post-Training)

## Using SageMaker Processing Jobs

In [ ]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r training_job_name

In [ ]:
try:
    training_job_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

In [ ]:
print(training_job_name)

# Review Data

In [ ]:
import pandas as pd

data = pd.read_json('data-clarify/test_bias.jsonl', lines=True)
data.shape

In [ ]:
data.head()

# Create Model

In [ ]:
import sagemaker

inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.3.1",
    py_version="py37",
    instance_type='ml.m5.4xlarge',
    image_scope="inference"
)
print(inference_image_uri)

In [ ]:
model_name = sess.create_model_from_job(
    training_job_name=training_job_name,
    image_uri=inference_image_uri
)
print(model_name)

# Detecting Bias with Amazon SageMaker Clarify

SageMaker Clarify helps you detect possible pre- and post-training biases using a variety of metrics.

## We need use a patch image to support JSONLines correctly for now
                         
```clarify_processor = clarify.SageMakerClarifyProcessor(...
        clarify_processor.image_uri = xxx```

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.c5.2xlarge',
                                                      sagemaker_session=sess)

# patch image 
clarify_processor.image_uri = "678264136642.dkr.ecr.us-east-1.amazonaws.com/sagemaker-clarify-processing:1.0_jsonlines_patch"

# Post-training Bias
Computing post-training bias metrics does require a trained model.

Unbiased training data (as determined by concepts of fairness measured by bias metric) may still result in biased model predictions after training. Whether this occurs depends on several factors including hyperparameter choices.


You can run these options separately with `run_pre_training_bias()` and `run_post_training_bias()` or at the same time with `run_bias()` as shown below.

# Writing DataConfig and ModelConfig
A `DataConfig` object communicates some basic information about data I/O to Clarify. We specify where to find the input dataset, where to store the output, the target column (`label`), the header names, and the dataset type.

Similarly, the `ModelConfig` object communicates information about your trained model and `ModelPredictedLabelConfig` provides information on the format of your predictions.  

**Note**: To avoid additional traffic to your production models, SageMaker Clarify sets up and tears down a dedicated endpoint when processing. `ModelConfig` specifies your preferred instance type and instance count used to run your model on during Clarify's processing.

## Upload Data To S3

In [ ]:
path = './data-clarify/test_bias.jsonl'

In [ ]:
!head -n 5 $path

In [ ]:
test_bias_path_jsonlines_s3_uri = sess.upload_data(bucket=bucket, key_prefix=training_job_name, path=path)
test_bias_path_jsonlines_s3_uri

In [ ]:
%store test_bias_path_jsonlines_s3_uri

# Configure Clarify

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, training_job_name)

data_config = clarify.DataConfig(s3_data_input_path=test_bias_path_jsonlines_s3_uri,
                                 s3_output_path=bias_report_output_path,
                                 label='star_rating',
                                 features='features',
                                 # label must be last, features in exact order as passed into model
                                 headers=['review_body', 'product_category', 'star_rating'],
                                 dataset_type='application/jsonlines')

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.4xlarge',
                                   instance_count=1,
                                   content_type='application/jsonlines',
                                   accept_type='application/jsonlines',
                                   # {"features": ["the worst", "Digital_Software"]}
                                   content_template='{"features":$features}'
                                  )

## _Note: `label` is set to the JSON key for the model prediction results_

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(label='predicted_label')

# Run Clarify

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[5,4], # needs to be int or str for continuous dtype, needs to be >1 for categorical dtype
                                facet_name='product_category',
#                                facet_values_or_threshold=['Gift Card'],
                                group_name='product_category')

In [ ]:
clarify_processor.run_post_training_bias(
    data_config=data_config,
    data_bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
#    methods='all', # FlipTest requires all columns to be numeric
    methods=['DPPL', 'DI', 'DCA', 'DCR', 'RD', 'DAR', 'DRR', 'AD', 'CDDPL', 'TE'],
    wait=False,
    logs=False)

In [ ]:
run_post_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_post_training_bias_processing_job_name

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, run_post_training_bias_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, run_post_training_bias_processing_job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, run_post_training_bias_processing_job_name, region)))


In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=run_post_training_bias_processing_job_name,
                                                                            sagemaker_session=sess)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
running_processor.wait(logs=False)

#### Viewing the Bias Report
In Studio, you can view the results under the experiments tab.

<img src="img/bias_report.gif">

Each bias metric has detailed explanations with examples that you can explore.

<img src="img/bias_detail.gif">

You could also summarize the results in a handy table!

<img src="img/bias_report_chart.gif">


If you're not a Studio user yet, you can access the bias report in pdf, html and ipynb formats in the following S3 bucket:

# Download Report From S3

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))


# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}